This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
 

In [1]:
figname = '../figures/fig_crowding'

In [2]:
%run 0_parameters.ipynb


This notebook introduces the problem addressed in this paper:

 - localizating an object in a large image
 - foveation
 - action (saccade)
       
      
{'w': 28, 'minibatch_size': 100, 'train_batch_size': 50000, 'test_batch_size': 5000, 'noise_batch_size': 1000, 'mean': 0.1307, 'std': 0.3081, 'N_pic': 128, 'offset_std': 30, 'offset_max': 34, 'noise': 1.0, 'contrast': 1.0, 'sf_0': 0.2, 'B_sf': 0.3, 'N_theta': 6, 'N_azimuth': 26, 'N_eccentricity': 10, 'N_phase': 2, 'rho': 1.41, 'bias_deconv': True, 'p_dropout': 0.0, 'dim1': 500, 'dim2': 2000, 'lr': 0.001, 'do_adam': True, 'bn1_bn_momentum': 0.5, 'bn2_bn_momentum': 0.2, 'momentum': 0.1, 'epochs': 25, 'num_processes': 1, 'no_cuda': True, 'log_interval': 100, 'verbose': 1, 'filename': '../data/2019-03-27', 'seed': 2019, 'N_cv': 8, 'do_compute': True}
Overwriting train.py
2019-07-12T22:26:41+02:00

CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
matplotlib 3.1.1
torch 1.1.0

compiler   : Clang 10.0.1 (clang-1001.0.46.3)
system     : Darwin
rele

# Training the WhatWhere network

In [3]:
%ls -lh ../data/*pt

-rw-r--r--  1 laurentperrinet  staff    12M Mar 13 22:14 ../data/2019-03-13_train.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 14 17:44 ../data/2019-03-14_b_train.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 14 06:17 ../data/2019-03-14_train.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 14 14:54 ../data/2019-03-14_train3.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 14 19:28 ../data/2019-03-14_train4.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 15 22:18 ../data/2019-03-15_train.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 15 15:31 ../data/2019-03-15_train_b.pt
-rw-r--r--  1 laurentperrinet  staff   8.8M Mar 20 17:46 ../data/2019-03-16_train.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 19 00:19 ../data/2019-03-18_train.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 19 15:18 ../data/2019-03-19_bis_train.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 19 15:54 ../data/2019-03-19_train.pt
-rw-r--r--  1 laurentperrinet  staff    12M Mar 27 16:3

In [4]:
%ls -lh ../data/*npy

-rw-r--r--  1 laurentperrinet  staff   800B Mar 14 15:57 ../data/2019-02-13-add-crowding-test-continued-result.npy
-rw-r--r--  1 laurentperrinet  staff   880B Mar 14 15:57 ../data/2019-02-13-add-crowding-test-result.npy
-rw-r--r--  1 laurentperrinet  staff    12M Mar  6 09:41 ../data/2019-02-15-anywhere-additive-noise-white-1000-Necc-10.npy
-rw-r--r--  1 laurentperrinet  staff    12M Mar  6 19:18 ../data/2019-03-06_CNS.npy
-rw-r--r--  1 laurentperrinet  staff    12M Mar  7 20:31 ../data/2019-03-07_CNS.npy
-rw-r--r--  1 laurentperrinet  staff    12M Mar  8 16:16 ../data/2019-03-08_CNS.npy
-rw-r--r--  1 laurentperrinet  staff    12M Mar  9 03:35 ../data/2019-03-11_CNS.npy
-rw-r--r--  1 laurentperrinet  staff    16K Apr  3 12:05 ../data/2019-03-29-crowding-result.npy
-rw-r--r--  1 laurentperrinet  staff    24K Jul  4 14:43 ../data/AccuracyMap_--MNIST_cnn_robust_what_0.1_0.1_1.0_0.7_5epoques_2019-06-19_11h50--_2019-06-19_12h23.npy
-rw-r--r--  1 laurentperrinet  staff    24K Jul  4 14:43 ..

In [5]:
from main_orig import init
#args = init(filename='debug')
args = init(filename='../data/2019-03-28')
from whatWhere import WhatWhere
whatWhere = WhatWhere(args, save = True)
print('starting training...')
filename_train = args.filename + '_train.pt'
#filename_train = "../data/2019-03-14_train4.pt"
#filename_train = "../data/2019-03-15_train_b.pt"
%ls -lh {filename_train}
#%rm {filename_train}
#%rm  ../data/debug_train.pt

whatWhere.train(filename_train)

ModuleNotFoundError: No module named 'main_orig'

In [ ]:
data_d, label_d = next(iter(whatWhere.display.loader_test))
N_plot = 5
fig, axs = plt.subplots(1, N_plot, figsize = (21, 5))
for i_ax, ax in enumerate(axs):
    ax.imshow(data_d[i_ax+5, 0, :, :], cmap=plt.gray())
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
N_plot = 3
fig, axs = plt.subplots(1, N_plot, figsize = (21, 8))
for i_ax, ax in enumerate(axs):
    data_fullfield, i_offset, j_offset = whatWhere.display.draw(data_d[i_ax, 0, :, :].numpy())
    ax = whatWhere.display.show(ax, data_fullfield)


In [ ]:
from main_orig import init
#args = init(filename='debug')
args = init(filename='../data/2019-03-28')
data, label = next(iter(whatWhere.loader_train)) 
print(data.shape)
print(label.shape)
pred = whatWhere.model.forward(data).argmax(dim=1, keepdim=True)
print(pred.eq(label.view_as(pred)).sum())
print(whatWhere.pred_accuracy(data, label)) #/whatWhere.args.minibatch_size)


In [ ]:
data, label = next(iter(whatWhere.loader_test)) 
print(data.shape)
print(label.shape)
pred = whatWhere.model.forward(data).argmax(dim=1, keepdim=True)
print(pred.eq(label.view_as(pred)).sum())
print(whatWhere.pred_accuracy(data, label))


In [ ]:
N_plot = 8
fig, axs = plt.subplots(1, 4, figsize = (21, 8))
for i_ax, ax in enumerate(axs):
    num = i_ax+20
    data_fullfield = whatWhere.retina.retina_invert(data[num,:].numpy())
    ax = whatWhere.retina.show(ax, data_fullfield)
    print(ax)
    print(label[num].numpy()) #ax.title(label[i_ax].numpy())
    print(pred[num]) #ax.title(label[i_ax].numpy())

In [ ]:
print(f'sf_0 : {args.sf_0}, B_sf : {args.B_sf}, acc : {acc}')

In [ ]:
result[period] = {} 

In [ ]:
result[period][prop] = acc

FIC_NAME = '../data/2019-03-29-crowding-test'
np.save(FIC_NAME + '-result.npy', result)

res = np.load(FIC_NAME + '-result.npy')
print(res)

In [ ]:
from retina import Display


FIC_NAME = '../data/2019-03-29-crowding-result.npy'
res = np.load(FIC_NAME).tolist()
shape = len(res), len(res[1])
im = np.zeros(shape)
for i, period in enumerate(range(1,21)):
    for j, prop in enumerate(np.arange(0.25,2.25,0.25)):
        im[i, j] = res[period][prop]
        
FIC_NAME = '../data/2019-04-02-crowding-contrast-result.npy'
res = np.load(FIC_NAME).tolist()
shape = len(res), len(res[1])
im2 = np.zeros(shape)
for i, period in enumerate(range(1,21)):
    for j, prop in enumerate((0.5, 0.7)):
        im2[i, j] = res[period][prop]

fig = plt.figure(figsize = (10,10))

period = 2
args.sf_0 = 1/period
args.B_sf = 1/period
args.contrast = 0.5
ax = fig.add_subplot(3, 3, 1)
ax.set_title(f'Period = {period}, contrast = {args.contrast}', fontsize = 14)
display = Display(args, save = False)
data, _ = next(iter(display.loader_test))
data_fullfield, i_offset, j_offset = display.draw(data[4, 0, :, :].numpy(), i_offset=-15, j_offset=15)
ax = display.show(ax, data_fullfield)

period = 10
args.sf_0 = 1/period
args.B_sf = 1/period
args.contrast = 0.5
ax = fig.add_subplot(3, 3, 2)
ax.set_title(f'Period = {period}, contrast = {args.contrast}', fontsize = 14)
display = Display(args, save = False)
data_fullfield, i_offset, j_offset = display.draw(data[4, 0, :, :].numpy(), i_offset=-15, j_offset=15)
ax = display.show(ax, data_fullfield)

period = 18
args.sf_0 = 1/period
args.B_sf = 1/period
args.contrast = 0.5
ax = fig.add_subplot(3, 3, 3)
ax.set_title(f'Period = {period}, contrast = {args.contrast}', fontsize = 14)
display = Display(args, save = False)
data_fullfield, i_offset, j_offset = display.draw(data[4, 0, :, :].numpy(), i_offset=-15, j_offset=15)
ax = display.show(ax, data_fullfield)

ax = fig.add_subplot(3, 1, (2,3))
#ax.fill_between(range(1,21), im[:,3], color = 'b', alpha = .3)
ax.plot(range(1,21), im[:,3], linewidth = 3, label = 'contrast = 0.3')
ax.plot(range(1,21), im2[:,0], linewidth = 3, label = 'contrast = 0.5')
ax.plot(range(1,21), im2[:,1], linewidth = 3, label = 'contrast = 0.7')
_ = ax.set_xticks(range(1,21))
_ = ax.set_xlabel('Noise spatial period (pixels)', fontsize = 18)
_ = ax.set_xlim([1, 20])
_ = ax.set_ylabel('Class accuracy', fontsize = 18)
_ = ax.set_ylim([0, 0.9])
_ = ax.plot([1, 20],[0.1, 0.1], ':')
_ = ax.legend()



In [ ]:
args